<a href="https://colab.research.google.com/github/xxotto/covid19-peru/blob/main/Notebooks/3_DataFiltering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Obteniendo más información de cada caso positivo por COVID-19 en Perú

En este notebook se busca obtener un dataset de todos los **casos positivos** reportados por COVID-19, y agregar información sobre fallecimiento y vacunación recibida (mediante **"UUID" o "id_persona"**). Es decir se busca unir las 3 bases de datos, de casos positivos, fallecimientos y vacunación recibida en una sola.

Para el manejo de información se siguen las pautas de 
"**STR**engthening **A**nalytical **T**hinking for **O**bservational **S**tudies" (STRATOS) escencialmente en [The STRATOS Iniciative - Topic Group 3](https://stratos-initiative.org/group_3).

- I. Metadata setup
- II. Data cleaning
- III. Data screening
- IV. Initial data reporting
- V. Refining and updating the analysis plan
- VI. Reporting IDA in research papers

## 1. Metadata setup

### 1.1 Descargar fuentes de datos

Las fuentes de datos pueden ser descargadas a través de la [Plataforma Nacional de Datos Abiertos](https://www.datosabiertos.gob.pe/) del Gobierno del Perú.

1. [Casos positivos por COVID-19](https://www.datosabiertos.gob.pe/dataset/casos-positivos-por-covid-19-ministerio-de-salud-minsa)
2. [Fallecidos por COVID-19](https://www.datosabiertos.gob.pe/dataset/fallecidos-por-covid-19-ministerio-de-salud-minsa)
3. [Vacunación contra la COVID-19](https://www.datosabiertos.gob.pe/dataset/vacunacion)

**Nota:** Puede que existan cambios en las bases de datos usadas ya que son actualizadas a diario (actualmente), por tanto cambios en el código pueden ser necesarios más adelante.

## 1.2 Montar directorio de trabajo

En este caso el directorio de trabajo donde se almacenarán las bases de datos que acaba de descargar en la sección 1.1 y el resultado final del coolab, estarán ubicados en su propia cuenta de google drive. 

### 1.2.1 Acceder a su cuenta de google drive

Ejecutar la siguiente celda, y acceder mediante su cuenta de google para conectar coolab con su google drive.

In [1]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


Crear un directorio en google drive llamado "covid19-peru".

In [2]:
%cd /content/drive/MyDrive/

%mkdir -p covid19-peru
%cd covid19-peru

/content/drive/MyDrive
/content/drive/MyDrive/covid19-peru


## 1.2.2 Ubicar las bases de datos descargadas en el directorio de trabajo

In [3]:
%ls

DP3_covid19-peru_casos-positivos_+_info-fallecimiento_+_info-vacunas.csv
fallecidos_covid.csv
positivos_covid.csv
TB_VACUNACION_COVID19.csv


In [4]:
!p7zip -d TB_VACUNACION_COVID19.7z

/usr/bin/p7zip: cannot read TB_VACUNACION_COVID19.7z


In [5]:
%ls

DP3_covid19-peru_casos-positivos_+_info-fallecimiento_+_info-vacunas.csv
fallecidos_covid.csv
positivos_covid.csv
TB_VACUNACION_COVID19.csv


### 1.2 Instalar librerías necesarias

In [6]:
%pip install epiweeks

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## 2. Lectura y Procesamiento

### 2.2 Cargar Librerías

In [7]:
import pandas as pd
import numpy as np
from epiweeks import Week
from datetime import datetime

### 2.3 Diccionarios para codificar variables

In [8]:
dic_sex = {'MASCULINO': 1,'FEMENINO':2}
dic_met = {'PR': 1, 'PCR':2, 'AG':3}
dic_dep = {
    "AMAZONAS":1,
    "ANCASH":2,
    "APURIMAC":3,   
    "AREQUIPA":4,
    "AYACUCHO":5,   
    "CAJAMARCA":6,
    "CALLAO":7,
    "CUSCO":8,
    "HUANCAVELICA":9,	
    "HUANUCO":10,	
    "ICA":11,	
    "JUNIN":12,
    "LA LIBERTAD":13,
    "LAMBAYEQUE":14,
    "LIMA":15,
    "LORETO":16,
    "MADRE DE DIOS":17,
    "MOQUEGUA":18,	
    "PASCO":19,
    "PIURA":20,
    "PUNO":21,
    "SAN MARTIN":22,
    "TACNA":23,
    "TUMBES":24,
    "UCAYALI":25
}

### 2.4 Dataset de casos positivos

#### 2.4.1 Leer dataset

In [9]:
# Leer dataset
url_cas = 'positivos_covid.csv'
col_cas = ['FECHA_RESULTADO','EDAD', 'DEPARTAMENTO', 'SEXO', 'id_persona', 'METODODX']
df_cas = pd.read_csv(url_cas, sep = ';', usecols = col_cas, dtype={'EDAD': 'float64', 'FECHA_RESULTADO': 'float64'})

In [10]:
df_cas.head()

,DEPARTAMENTO,METODODX,EDAD,SEXO,FECHA_RESULTADO,id_persona
0,AREQUIPA,PCR,78.0,FEMENINO,20220108.0,13889713.0
1,AREQUIPA,AG,67.0,MASCULINO,20210628.0,13889735.0
2,LIMA,PR,23.0,MASCULINO,20210121.0,13889774.0
3,JUNIN,AG,24.0,FEMENINO,20220119.0,13889793.0
4,LIMA,PR,57.0,FEMENINO,20200902.0,13889909.0


In [11]:
n = len(df_cas)  # Variable para almacenar el tamaño de cada dataset luego de cada cambio
print(f'Número de casos positivos: {n:10} ')

Número de casos positivos:    3572423 


#### 2.4.2 Tratamiento de datos (Data cleaning)

In [12]:
df_cas.isnull().sum()  # Total de 'missing values' por columna

DEPARTAMENTO         321
METODODX               0
EDAD                 347
SEXO                   1
FECHA_RESULTADO     2023
id_persona         64148
dtype: int64

In [13]:
df_cas = df_cas.dropna()  # Remover casos con información faltante
df_cas.isnull().sum()  # Total de 'missing values' por columna

DEPARTAMENTO       0
METODODX           0
EDAD               0
SEXO               0
FECHA_RESULTADO    0
id_persona         0
dtype: int64

In [14]:
print(f'Missing values eliminados: {n - len(df_cas):10}')
print(f'Número de casos positivos: {len(df_cas):10} ')
n = len(df_cas)

Missing values eliminados:      66615
Número de casos positivos:    3505808 


In [15]:
df_cas['DEPARTAMENTO'].value_counts().sort_index()  # Analizar columna DEPARTAMENTO

AMAZONAS           43752
ANCASH            125834
APURIMAC           38960
AREQUIPA          213096
ARICA                  1
AYACUCHO           47934
CAJAMARCA          95668
CALLAO            142796
CUSCO             115816
HUANCAVELICA       25738
HUANUCO            51787
ICA               102362
JUNIN             125441
LA LIBERTAD       144460
LAMBAYEQUE        104791
LIMA             1578223
LORETO             57081
MADRE DE DIOS      17589
MOQUEGUA           48338
PASCO              23901
PIURA             156845
PUNO               65839
SAN MARTIN         61339
TACNA              53082
TUMBES             26877
UCAYALI            38258
Name: DEPARTAMENTO, dtype: int64

In [16]:
# Remover valores erróneos
rows_to_delete = df_cas[df_cas['DEPARTAMENTO'] == 'ARICA'].index
df_cas.drop(rows_to_delete, inplace = True)

In [17]:
df_cas['METODODX'].value_counts().sort_index()  # Analizar columna METODODX

AG     1498196
PCR    1067272
PR      940339
Name: METODODX, dtype: int64

In [18]:
# Analizar columna EDAD
edadCasos = df_cas['EDAD'].value_counts()

print('Edad (años): no_casos\n')
lst = []

for idx, edad in enumerate(edadCasos):
    index = edadCasos.index
    lst.append(f'{index[idx]:6}: {edad}   ')
    
lst = sorted(lst)
chunks = [lst[x:x+25] for x in range(0, len(lst), 25)]
res = pd.DataFrame.from_records(chunks).transpose()

print(res.to_string(index=False, header=False))

Edad (años): no_casos

    0.0: 8831      25.0: 80677      50.0: 55122      75.0: 14992       -79.0: 1   
    1.0: 8691      26.0: 83781      51.0: 52837      76.0: 13616     100.0: 158   
    2.0: 6768      27.0: 85937      52.0: 51490      77.0: 12663     101.0: 116   
    3.0: 6666      28.0: 88768      53.0: 50519      78.0: 11565      102.0: 76   
    4.0: 6894      29.0: 87744      54.0: 49890      79.0: 10376      103.0: 43   
    5.0: 7756      30.0: 84820      55.0: 47725       80.0: 9675      104.0: 20   
    6.0: 8821      31.0: 84901      56.0: 45410       81.0: 9037      105.0: 16   
    7.0: 9965      32.0: 87664      57.0: 43437       82.0: 8205      106.0: 11   
   8.0: 11128      33.0: 89579      58.0: 41290       83.0: 7546       107.0: 6   
   9.0: 11981      34.0: 83648      59.0: 39790       84.0: 6759       108.0: 8   
  10.0: 13543      35.0: 79785      60.0: 38604       85.0: 6044       109.0: 3   
  11.0: 15693      36.0: 77390      61.0: 37243       86.0: 5295

In [19]:
# Remover valores erróneos
rows_to_delete = df_cas[(df_cas['EDAD'] < 0) | (df_cas['EDAD'] > 122)].index
df_cas.drop(rows_to_delete, inplace = True)

In [20]:
df_cas['SEXO'].value_counts().sort_index()  # Analizar columna SEXO

FEMENINO     1770439
MASCULINO    1735362
Name: SEXO, dtype: int64

In [21]:
df_cas['FECHA_RESULTADO'].nunique()  # Analizar columna FECHA_RESULTADO (Número de días con reportes de casos)

798

In [22]:
personas_covid = df_cas['id_persona'].nunique()  # Analizar columna id_persona
personas_covid

3274218

In [23]:
print(f'Casos con valores erróneos eliminados: {n - len(df_cas):10}')
print(f'Número de casos positivos: {len(df_cas):10} ')
n = len(df_cas)

Casos con valores erróneos eliminados:          7
Número de casos positivos:    3505801 


In [24]:
# Ordenar columnas
df_cas.rename(columns = {'FECHA_RESULTADO':'fecha_resultado',
                         'EDAD': 'edad',
                         'SEXO': 'sexo',
                         'DEPARTAMENTO': 'departamento',
                         'METODODX': 'metododx'}, inplace=True)

df_cas = df_cas[['id_persona','fecha_resultado', 'metododx', 'sexo', 'edad', 'departamento']] \
         .sort_values(by = ['fecha_resultado']) \
         .reset_index(drop = True)

In [25]:
# Codificar variables y cambiar 'data types'
df_cas['sexo'].replace(dic_sex, inplace=True)
df_cas['departamento'] = df_cas['departamento'].map(dic_dep)
df_cas['metododx'].replace(dic_met, inplace=True)

In [26]:
# Cambiar 'data type' para reducir tamaño
df_cas = df_cas.astype({'id_persona': 'int32', 'fecha_resultado': 'int32', 'edad':'int8', 'departamento': 'int8',
                        'metododx': 'int8', 'sexo': 'int8'})

In [27]:
df_cas.head()

,id_persona,fecha_resultado,metododx,sexo,edad,departamento
0,34978840,20200306,2,1,25,15
1,10629488,20200307,2,1,51,15
2,7800643,20200307,2,2,74,15
3,12211040,20200307,2,1,7,15
4,29640291,20200307,2,1,29,4


#### 2.4.3 Eliminar registros duplicados

Se decide eliminar casos de registros duplicados. Es decir eliminar del dataset todas las reinnfecciones de COVID-19 y conservar únicamente la última reinfección en caso de registros duplicados en id_persona.

In [28]:
# Ordenar de acuerdo al id_de persona y fecha de resultado
df_cas = df_cas.sort_values(by = ['id_persona', 'fecha_resultado'])

In [29]:
# Vemos algunos de los registros duplicados
df_cas[df_cas.id_persona.duplicated() == True].head()

,id_persona,fecha_resultado,metododx,sexo,edad,departamento
3201447,47,20220129,2,1,62,25
3200873,89,20220129,2,2,74,25
1733928,227,20210421,3,1,81,25
1919698,324,20210525,3,2,54,25
3152919,555,20220127,2,2,55,25


In [30]:
# Comprobamos duplicados en el dataset principal de casos (df_cas)
df_cas[df_cas['id_persona'] == 47]

,id_persona,fecha_resultado,metododx,sexo,edad,departamento
333420,47,20200711,1,1,61,25
3201447,47,20220129,2,1,62,25


En este último caso el **id_persona** número **47** dió positivo a COVID-19 dos veces. La primera el 2020-07-11 y la segunda en 2022-01-29. 

Para el motivo del estudio se necesita información de UNA SOLA PERSONA, por lo que se decide eliminar estos casos de reinfecciones y conservar solamente la última vez que dió positivo. 

Estos duplicados representan varios contagios de la misma persona, sin embargo se toman como un nuevo contagio en la base de datos original.

In [31]:
# Eliminamos los duplicados
df_cas = df_cas.drop_duplicates('id_persona', keep='last') \
               .sort_values(by = ['fecha_resultado']) \
               .reset_index(drop = True)

In [32]:
df_cas.head()

,id_persona,fecha_resultado,metododx,sexo,edad,departamento
0,13708801,20200309,2,2,15,10
1,2020432,20200310,2,1,46,15
2,7979960,20200311,2,2,21,15
3,5493803,20200311,2,2,28,15
4,36611013,20200312,2,1,78,15


In [33]:
# TOTAL de casos positivos con un id único para cada persona (sin registros duplicados)
len(df_cas)

3274218

In [34]:
print(f'Casos de reinfecciones eliminados: {n - len(df_cas):10}')
print(f'Número de casos positivos: {len(df_cas):10} ')
n = len(df_cas)

Casos de reinfecciones eliminados:     231583
Número de casos positivos:    3274218 


### 2.5 Dataset de fallecidos

#### 2.5.1 Leer dataset

In [35]:
# Leer el dataset
url_fal = 'fallecidos_covid.csv'
col_fal = ['FECHA_FALLECIMIENTO', 'UUID', 'SEXO', 'EDAD_DECLARADA', 'DEPARTAMENTO']
df_fal = pd.read_csv(url_fal, sep = ';', usecols= col_fal, dtype = {'FECHA_FALLECIMIENTO':'int32'}, low_memory=False)

In [36]:
df_fal.head()

,FECHA_FALLECIMIENTO,EDAD_DECLARADA,SEXO,DEPARTAMENTO,UUID
0,20210512,88,FEMENINO,AREQUIPA,13889730.0
1,20210401,72,FEMENINO,AREQUIPA,13889746.0
2,20210624,58,MASCULINO,AREQUIPA,13889749.0
3,20210426,84,MASCULINO,LA LIBERTAD,13889932.0
4,20210315,64,MASCULINO,LA LIBERTAD,13889956.0


In [37]:
n = len(df_fal)  # TOTAL de fallecidos por COVID-19
n

213040

#### 2.5.2 Tratamiento de datos (Data cleaning)

In [38]:
df_fal.isnull().sum()  # Total de 'missing values' por columna

FECHA_FALLECIMIENTO       0
EDAD_DECLARADA            0
SEXO                      0
DEPARTAMENTO              0
UUID                   2603
dtype: int64

In [39]:
df_fal = df_fal.dropna()  # Remover casos con información faltante
df_fal.isnull().sum()  # Total de 'missing values' por columna

FECHA_FALLECIMIENTO    0
EDAD_DECLARADA         0
SEXO                   0
DEPARTAMENTO           0
UUID                   0
dtype: int64

In [40]:
print(f'TOTAL de casos fallecidos con missing values: {n - len(df_fal):10}')
print(f'Número de fallecidos: {len(df_fal):10}')
n = len(df_fal)

TOTAL de casos fallecidos con missing values:       2603
Número de fallecidos:     210437


In [41]:
# Renombrar columnas
df_fal.rename(columns = {
    'FECHA_FALLECIMIENTO':'fecha_fallecimiento',
    'UUID':'id_persona',
    'SEXO': 'sexo', 
    'EDAD_DECLARADA': 'edad',
    'DEPARTAMENTO': 'departamento'
    }, inplace=True)

In [42]:
# Analizar columna EDAD
edadCasos = df_fal['edad'].value_counts()

print('Edad (años): no_casos\n')
lst = []

for idx, edad in enumerate(edadCasos):
    index = edadCasos.index
    lst.append(f'{index[idx]:6}: {edad}   ')
    
lst = sorted(lst)
chunks = [lst[x:x+25] for x in range(0, len(lst), 25)]
res = pd.DataFrame.from_records(chunks).transpose()

print(res.to_string(index=False, header=False))

Edad (años): no_casos

     0: 138         25: 248        50: 2588        75: 5132       100: 128   
     1: 134         26: 275        51: 2741        76: 4993        101: 87   
      2: 75         27: 339        52: 2850        77: 4781        102: 54   
      3: 51         28: 323        53: 3148        78: 4640        103: 30   
      4: 55         29: 344        54: 3486        79: 4276        104: 18   
      5: 47         30: 409        55: 3753        80: 4365         105: 6   
      6: 46         31: 420        56: 3791        81: 4051        106: 12   
      7: 51         32: 568        57: 4061        82: 4113         107: 3   
      8: 47         33: 622        58: 4115        83: 3813         108: 6   
      9: 54         34: 647        59: 4377        84: 3689         109: 1   
     10: 54         35: 658        60: 4683        85: 3481         110: 2   
     11: 56         36: 690        61: 5145        86: 3101         113: 2   
     12: 47         37: 847        62: 50

In [43]:
df_fal['sexo'].value_counts().sort_index()  # Analizar columna SEXO

FEMENINO      77060
MASCULINO    133377
Name: sexo, dtype: int64

In [44]:
# Reemplazar sexo con un code
df_fal['sexo'].replace(dic_sex, inplace=True)
df_fal['sexo'] = df_fal['sexo'].astype('int8')

In [45]:
df_fal['departamento'].value_counts().sort_index()  # Analizar columna DEPARTAMENTO

AMAZONAS          1318
ANCASH            7115
APURIMAC          1629
AREQUIPA         10271
AYACUCHO          2321
CAJAMARCA         4452
CALLAO           10388
CUSCO             5147
HUANCAVELICA      1266
HUANUCO           2871
ICA               8883
JUNIN             7438
LA LIBERTAD      10796
LAMBAYEQUE        8988
LIMA             92051
LORETO            4176
MADRE DE DIOS      848
MOQUEGUA          1614
PASCO             1092
PIURA            12934
PUNO              4762
SAN MARTIN        3140
TACNA             2118
TUMBES            1677
UCAYALI           3142
Name: departamento, dtype: int64

In [46]:
# Codificar variable departamento y cambiar 'data types'
df_fal['departamento'] = df_fal['departamento'].map(dic_dep)
df_fal['departamento'] = df_fal['departamento'].astype('int8')

# Nombrar a todos los casos de fallecimientos con un identificador
df_fal['fallecido'] = 1

# Cambiar de data type para reducir tamaño
df_fal = df_fal.astype({'id_persona': 'int32', 'fallecido': 'int8', 'sexo': 'int8', 'edad': 'int8',
                        'fecha_fallecimiento': 'int32'})

# Transformar a formato fecha para operar los días más adelante
df_fal['fecha_fal'] = pd.to_datetime(df_fal['fecha_fallecimiento'], format = '%Y%m%d')

# Agregar la semana epidemiológica y el año
df_fal[['year','epi_week']] = df_fal['fecha_fal'].apply(lambda date : Week.fromdate(date).weektuple()).tolist()

In [47]:
df_fal.head()

,fecha_fallecimiento,edad,sexo,departamento,id_persona,fallecido,fecha_fal,year,epi_week
0,20210512,88,2,4,13889730,1,2021-05-12,2021,19
1,20210401,72,2,4,13889746,1,2021-04-01,2021,13
2,20210624,58,1,4,13889749,1,2021-06-24,2021,25
3,20210426,84,1,13,13889932,1,2021-04-26,2021,17
4,20210315,64,1,13,13889956,1,2021-03-15,2021,11


#### 2.4.3 Eliminar registros duplicados

Existen registros duplicados en el dataset de fallecidos, probablemente errores al momento de ingresar la información.

In [48]:
# Ordenar de acuerdo al id_de persona y fecha de fallecimiento
df_fal = df_fal.sort_values(by = ['id_persona', 'fecha_fallecimiento'])

In [49]:
# Vemos algunos de los registros duplicados
df_fal[df_fal.id_persona.duplicated() == True].head()

,fecha_fallecimiento,edad,sexo,departamento,id_persona,fallecido,fecha_fal,year,epi_week
29615,20210510,63,2,11,946577,1,2021-05-10,2021,19
33491,20200625,65,2,15,1378132,1,2020-06-25,2020,26
35007,20210407,60,1,15,1581994,1,2021-04-07,2021,14
36208,20210317,61,2,15,1676066,1,2021-03-17,2021,11
38636,20210323,64,1,15,1785991,1,2021-03-23,2021,12


In [50]:
# Comprobamos duplicados en el dataset principal de fallecidos (df_fal)
df_fal[df_fal['id_persona'] == 946577]

,fecha_fallecimiento,edad,sexo,departamento,id_persona,fallecido,fecha_fal,year,epi_week
29609,20210510,73,2,11,946577,1,2021-05-10,2021,19
29615,20210510,63,2,11,946577,1,2021-05-10,2021,19


In [51]:
# Eliminamos los duplicados
df_fal = df_fal.drop_duplicates('id_persona', keep='last') \
               .sort_values(by = ['fecha_fallecimiento']) \
               .reset_index(drop = True)

In [52]:
df_fal.head()

,fecha_fallecimiento,edad,sexo,departamento,id_persona,fallecido,fecha_fal,year,epi_week
0,20200303,63,1,22,36527082,1,2020-03-03,2020,10
1,20200304,85,1,22,36401995,1,2020-03-04,2020,10
2,20200307,24,2,16,36853977,1,2020-03-07,2020,10
3,20200308,76,2,8,14179222,1,2020-03-08,2020,11
4,20200311,81,1,15,38353395,1,2020-03-11,2020,11


In [53]:
# TOTAL de fallecidos con un id único para cada persona (sin registros duplicados)
print(f'TOTAL de duplicados: {n - len(df_fal):10}')
print(f'Número de fallecidos: {len(df_fal):10}')
n = len(df_fal)

TOTAL de duplicados:         71
Número de fallecidos:     210366


### 2.6 Buscar en el dataset de casos positivos quienes fallecieron y cuándo

In [54]:
# Unir el dataframe de casos positivos con el dataset de fallecidos mediante 'id_persona'
df = df_cas.merge(df_fal, on = 'id_persona', how = 'outer')

df['fallecido'].fillna(0, inplace=True)  # Llenar con 0 las personas que no fallecieron en el dataset de casos positivos
df['fallecido'] = df['fallecido'].astype('int8')  # Cambiar 'data type' para reducir tamaño

In [55]:
len(df)  # Total de casos de casos positivos + información de fallecimiento

3381496

In [56]:
len(df[df.fallecido == 1])  # Número de fallecidos en el dataset de casos postivos

210366

#### 2.6.1 Unir información de casos positivos + información sobre fallecimiento

In [57]:
df.head()

,id_persona,fecha_resultado,metododx,sexo_x,edad_x,departamento_x,fecha_fallecimiento,edad_y,sexo_y,departamento_y,fallecido,fecha_fal,year,epi_week
0,13708801,20200309.0,2.0,2.0,15.0,10.0,NaN,NaN,NaN,NaN,0,NaT,NaN,NaN
1,2020432,20200310.0,2.0,1.0,46.0,15.0,NaN,NaN,NaN,NaN,0,NaT,NaN,NaN
2,7979960,20200311.0,2.0,2.0,21.0,15.0,NaN,NaN,NaN,NaN,0,NaT,NaN,NaN
3,5493803,20200311.0,2.0,2.0,28.0,15.0,NaN,NaN,NaN,NaN,0,NaT,NaN,NaN
4,36611013,20200312.0,2.0,1.0,78.0,15.0,20200319.0,78.0,1.0,15.0,1,2020-03-19,2020.0,12.0


Tenemos algunas columnas duplicadas ya que son información del dataset de casos positivos (sexo_x, edad_x, departamento_x) y la otra información del dataset de fallecidos (sexo_y, edad_y, departamento_y). 

En este caso podemos ver que pertenecen a la misma persona ya que fueron unidos mediante el registro único de cada persona (id_persona). Procedemos a unir estas columnas duplicadas para completar información faltante.

In [58]:
df[['id_persona', 'sexo_x', 'edad_x', 'departamento_x',
    'sexo_y', 'edad_y', 'departamento_y', 'fallecido']][df['fallecido'] == 1].head()

,id_persona,sexo_x,edad_x,departamento_x,sexo_y,edad_y,departamento_y,fallecido
4,36611013,1.0,78.0,15.0,1.0,78.0,15.0,1
9,36889651,2.0,75.0,15.0,2.0,75.0,15.0,1
49,36450848,1.0,96.0,15.0,1.0,96.0,15.0,1
58,36539210,1.0,71.0,7.0,1.0,71.0,15.0,1
110,36764997,1.0,45.0,15.0,1.0,45.0,15.0,1


In [59]:
df.isnull().sum()  # Verificamos 'missing values'

id_persona                   0
fecha_resultado         107278
metododx                107278
sexo_x                  107278
edad_x                  107278
departamento_x          107278
fecha_fallecimiento    3171130
edad_y                 3171130
sexo_y                 3171130
departamento_y         3171130
fallecido                    0
fecha_fal              3171130
year                   3171130
epi_week               3171130
dtype: int64

In [60]:
# Agregamos la información de sexo, edad y departamento de fallecimientos (columnas y)
# a las columnas de casos positivos (columnas x)
df['sexo_x'] = df['sexo_x'].combine_first(df['sexo_y'])
df['edad_x'] = df['edad_x'].combine_first(df['edad_y'])
df['departamento_x'] = df['departamento_x'].combine_first(df['departamento_y'])

In [61]:
df.isnull().sum()  # Verificamos 'missing values'

id_persona                   0
fecha_resultado         107278
metododx                107278
sexo_x                       0
edad_x                       0
departamento_x               0
fecha_fallecimiento    3171130
edad_y                 3171130
sexo_y                 3171130
departamento_y         3171130
fallecido                    0
fecha_fal              3171130
year                   3171130
epi_week               3171130
dtype: int64

In [62]:
df

,id_persona,fecha_resultado,metododx,sexo_x,edad_x,departamento_x,fecha_fallecimiento,edad_y,sexo_y,departamento_y,fallecido,fecha_fal,year,epi_week
0,13708801,20200309.0,2.0,2.0,15.0,10.0,NaN,NaN,NaN,NaN,0,NaT,NaN,NaN
1,2020432,20200310.0,2.0,1.0,46.0,15.0,NaN,NaN,NaN,NaN,0,NaT,NaN,NaN
2,7979960,20200311.0,2.0,2.0,21.0,15.0,NaN,NaN,NaN,NaN,0,NaT,NaN,NaN
3,5493803,20200311.0,2.0,2.0,28.0,15.0,NaN,NaN,NaN,NaN,0,NaT,NaN,NaN
4,36611013,20200312.0,2.0,1.0,78.0,15.0,20200319.0,78.0,1.0,15.0,1,2020-03-19,2020.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3381491,6858518,NaN,NaN,1.0,80.0,18.0,20220511.0,80.0,1.0,18.0,1,2022-05-11,2022.0,19.0
3381492,745066,NaN,NaN,2.0,80.0,21.0,20220512.0,80.0,2.0,21.0,1,2022-05-12,2022.0,19.0
3381493,26737458,NaN,NaN,1.0,55.0,11.0,20220513.0,55.0,1.0,11.0,1,2022-05-13,2022.0,19.0
3381494,88155,NaN,NaN,1.0,82.0,24.0,20220514.0,82.0,1.0,24.0,1,2022-05-14,2022.0,19.0


In [63]:
# Renombramos y ordenamos columnas
df = df.rename(columns = {'sexo_x': 'sexo', 'edad_x': 'edad', 'departamento_x': 'departamento'})
df = df[['id_persona', 'fecha_resultado', 'metododx', 'sexo', 'edad', 'departamento',
         'fallecido', 'fecha_fallecimiento', 'fecha_fal', 'year', 'epi_week']]

In [64]:
df.head()

,id_persona,fecha_resultado,metododx,sexo,edad,departamento,fallecido,fecha_fallecimiento,fecha_fal,year,epi_week
0,13708801,20200309.0,2.0,2.0,15.0,10.0,0,NaN,NaT,NaN,NaN
1,2020432,20200310.0,2.0,1.0,46.0,15.0,0,NaN,NaT,NaN,NaN
2,7979960,20200311.0,2.0,2.0,21.0,15.0,0,NaN,NaT,NaN,NaN
3,5493803,20200311.0,2.0,2.0,28.0,15.0,0,NaN,NaT,NaN,NaN
4,36611013,20200312.0,2.0,1.0,78.0,15.0,1,20200319.0,2020-03-19,2020.0,12.0


In [65]:
# Obtener el número de días a partir del inicio de la pandemia en que una persona fallece
def days_between(d1, d2):
    '''Función que retorna el número de días entre 2 días dados'''
    return abs((d2 - d1).days)

fecha_min = df['fecha_fal'].min()
print(f'Primera fecha reportada de un fallecimiento en el dataset: {fecha_min}')
fecha_max = df['fecha_fal'].max()
print(f'Última fecha reportada de un fallecimiento el dataset: {fecha_max}')

# Obtener el número de días transcurridos a partir de la primera fecha reportada (fecha_min)
df['tiempo_fal'] = df['fecha_fal'].apply(lambda row : days_between(row, fecha_min)+1)

# Llenar 'nan values' con la última fecha del estudio
#df['tiempo_fal'] = df['tiempo_fal'].fillna(df['tiempo_fal'].max()).astype('int')

Primera fecha reportada de un fallecimiento en el dataset: 2020-03-03 00:00:00
Última fecha reportada de un fallecimiento el dataset: 2022-05-14 00:00:00


**NOTA**: Se toma como fecha cero la de fallecimiento porque el primer reporte de caso confirmado se da luego del reporte primer reporte de fallecimiento por COVID-19.

Ahora se busca transformar los días de fallecimientos (tiempo_fal) en semanas, para mostrar datos agrupados de una mejor manera.

In [66]:
df.fecha_fal.min()  # fecha de primer fallecimiento 3 de marzo del 2020

Timestamp('2020-03-03 00:00:00')

In [67]:
import calendar

print(calendar.month(2020, 3))

     March 2020
Mo Tu We Th Fr Sa Su
                   1
 2  3  4  5  6  7  8
 9 10 11 12 13 14 15
16 17 18 19 20 21 22
23 24 25 26 27 28 29
30 31



Lo que se necesita es agregar nuestra semana 1 desde el reporte del primer fallecido, es decir el 3 de marzo del 2020.

- Día 2 al 8 de marzo   | semana 1
- Día 9 al 15 de marzo  | semana 2
- Día 16 al 22 de marzo | semana 3
- Día 23 al 29 de marzo | semana 4
- ...

In [103]:
week_equivalent = {}  # Diccionario con la equivalencia de semana para cada día de fallecimiento
i = 0
week = 1

for day in range(0, int(df.tiempo_fal.max()) + 1):
  week_equivalent.update({day: week})
  i += 1
  if i%7 == 0:
    week += 1

Generar una nueva columna con nuestro diccionario que agrega la semana correspondiente para cada día de fallecimiento.

In [104]:
df['semana_fal'] = df['tiempo_fal'].map(week_equivalent)  # Columna con la semana de fallecimiento

In [105]:
df[df.fallecido == 1].head()

,id_persona,fecha_resultado,metododx,sexo,edad,edad_cat,departamento,fallecido,fecha_fallecimiento,fecha_fal,year,epi_week,tiempo_fal,semana_fal
4,36611013,20200312.0,2.0,1,78,6,15,1,20200319.0,2020-03-19,2020.0,12.0,17.0,3.0
9,36889651,20200312.0,2.0,2,75,6,15,1,20200320.0,2020-03-20,2020.0,12.0,18.0,3.0
49,36450848,20200315.0,2.0,1,96,7,15,1,20200329.0,2020-03-29,2020.0,14.0,27.0,4.0
58,36539210,20200316.0,2.0,1,71,6,7,1,20200414.0,2020-04-14,2020.0,16.0,43.0,7.0
110,36764997,20200318.0,2.0,1,45,3,15,1,20200319.0,2020-03-19,2020.0,12.0,17.0,3.0


In [106]:
# Categorizar edad en grupos etáreos
def edad_cat(df, col):
    """Función para categorizar edad en grupos etáreos"""
    conditions = [(df[col]< 18),
                  (df[col]>=18) & (df[col]<30),
                  (df[col]>=30) & (df[col]<40),
                  (df[col]>=40) & (df[col]<50),
                  (df[col]>=50) & (df[col]<60),
                  (df[col]>=60) & (df[col]<70),
                  (df[col]>=70) & (df[col]<80),
                  (df[col]>=80)]
    choices = [0,1,2,3,4,5,6,7]
    return np.select(conditions, choices, default=np.nan).astype('int8')

df['edad_cat'] = edad_cat(df, 'edad') 

In [107]:
# Ordenar columnas
df = df[['id_persona', 'fecha_resultado', 'metododx', 'sexo', 'edad', 'edad_cat',
       'departamento', 'fallecido', 'fecha_fallecimiento', 'fecha_fal', 'year',
       'epi_week', 'tiempo_fal', 'semana_fal']]

In [108]:
# Transformar 'data types'
df = df.astype({'sexo': 'int8', 'edad': 'int8', 'departamento': 'int8'})

In [109]:
df.head()

,id_persona,fecha_resultado,metododx,sexo,edad,edad_cat,departamento,fallecido,fecha_fallecimiento,fecha_fal,year,epi_week,tiempo_fal,semana_fal
0,13708801,20200309.0,2.0,2,15,0,10,0,NaN,NaT,NaN,NaN,NaN,NaN
1,2020432,20200310.0,2.0,1,46,3,15,0,NaN,NaT,NaN,NaN,NaN,NaN
2,7979960,20200311.0,2.0,2,21,1,15,0,NaN,NaT,NaN,NaN,NaN,NaN
3,5493803,20200311.0,2.0,2,28,1,15,0,NaN,NaT,NaN,NaN,NaN,NaN
4,36611013,20200312.0,2.0,1,78,6,15,1,20200319.0,2020-03-19,2020.0,12.0,17.0,3.0


In [110]:
# Missing values
df.isnull().sum()

id_persona                   0
fecha_resultado         107278
metododx                107278
sexo                         0
edad                         0
edad_cat                     0
departamento                 0
fallecido                    0
fecha_fallecimiento    3171130
fecha_fal              3171130
year                   3171130
epi_week               3171130
tiempo_fal             3171130
semana_fal             3171130
dtype: int64

Las columnas de fecha resultado y metododx con datos faltantes se deben a que se agregaron personas del dataset de fallecidos que no se encontraban en el dataset de casos positivos, es decir nunca se hicieron pruebas para confirmar COVID-19, pero están registrados como causa de muerte COVID-19, evidentemente son casos postivos y se agregan al dataset.

#### 2.6.1 Tratar 'missing values' en fechas de fallecimiento

Como no existe año (year) y semana epidemiológica (epi_week) de fallecimiento para las personas que NO fallecieron, se decide llenar estos espacios con el año y semana epidemiológica última reportada en el dataset.

In [111]:
# Tupla con año y semana epidemiológica última reportada en el dataset df
last_date = Week.fromdate(fecha_max).weektuple()
last_date

(2022, 19)

In [112]:
df['year'].fillna(last_date[0], inplace=True)
df['epi_week'].fillna(last_date[1], inplace=True)

# Transformar 'data types'
df = df.astype({'year': 'int16', 'epi_week': 'int8'})

In [113]:
df.head()

,id_persona,fecha_resultado,metododx,sexo,edad,edad_cat,departamento,fallecido,fecha_fallecimiento,fecha_fal,year,epi_week,tiempo_fal,semana_fal
0,13708801,20200309.0,2.0,2,15,0,10,0,NaN,NaT,2022,19,NaN,NaN
1,2020432,20200310.0,2.0,1,46,3,15,0,NaN,NaT,2022,19,NaN,NaN
2,7979960,20200311.0,2.0,2,21,1,15,0,NaN,NaT,2022,19,NaN,NaN
3,5493803,20200311.0,2.0,2,28,1,15,0,NaN,NaT,2022,19,NaN,NaN
4,36611013,20200312.0,2.0,1,78,6,15,1,20200319.0,2020-03-19,2020,12,17.0,3.0


In [114]:
# Missing values
df.isnull().sum()

id_persona                   0
fecha_resultado         107278
metododx                107278
sexo                         0
edad                         0
edad_cat                     0
departamento                 0
fallecido                    0
fecha_fallecimiento    3171130
fecha_fal              3171130
year                         0
epi_week                     0
tiempo_fal             3171130
semana_fal             3171130
dtype: int64

### 2.7 Dataset de vacunados

#### 2.7.1 Leer dataset

In [115]:
url_vac = 'TB_VACUNACION_COVID19.csv'
col_vac = ['id_persona','fecha_vacunacion','dosis', 'id_vacuna']
df_vac = pd.read_csv(url_vac, sep = ",", usecols = col_vac, dtype = {'fecha_vacunacion':'int32', 
                                                                      'dosis': 'int8',
                                                                      'id_vacuna': 'int8',
                                                                      'id_persona': 'int32'})

In [116]:
df_vac.head()

,id_persona,fecha_vacunacion,id_vacuna,dosis
0,22022061,20210929,6,2
1,29311935,20210519,5,2
2,28793138,20220209,5,3
3,33809596,20210821,6,2
4,33804613,20210721,6,1


In [117]:
# Renombrar y ordenar columnas
df_vac = df_vac.rename(columns = {'fecha_vacunacion': 'fecha_vacuna'})
df_vac = df_vac[['id_persona', 'dosis', 'fecha_vacuna', 'id_vacuna']]

In [118]:
df_vac.head()

,id_persona,dosis,fecha_vacuna,id_vacuna
0,22022061,2,20210929,6
1,29311935,2,20210519,5
2,28793138,3,20220209,5
3,33809596,2,20210821,6
4,33804613,1,20210721,6


In [119]:
# TOTAL de vacunas aplicadas
len(df_vac)

73605300

#### 2.7.2 Separar información de dosis recibidas para juntarlos con nuestro dataframe principal (df)

In [120]:
print(f'Total de dosis 1 administradas: {len(df_vac[df_vac.dosis == 1])}')
print(f'Total de dosis 2 administradas: {len(df_vac[df_vac.dosis == 2])}')
print(f'Total de dosis 3 administradas: {len(df_vac[df_vac.dosis == 3])}')
print(f'Total de 4 o más dosis administradas: {len(df_vac[df_vac.dosis >= 4])}')

Total de dosis 1 administradas: 29249219
Total de dosis 2 administradas: 27114642
Total de dosis 3 administradas: 16799594
Total de 4 o más dosis administradas: 441845


In [121]:
# Agregar información de dosis 1
df = df.merge(df_vac[df_vac.dosis == 1], on = 'id_persona', how = 'left')
df = df.rename(columns = {'dosis' : 'dosis_1', 
                          'fecha_vacuna': 'fecha_vacuna_1', 
                          'id_vacuna': 'id_vacuna_1'})

In [122]:
# Agregar información de dosis 2
df = df.merge(df_vac[df_vac.dosis == 2], on = 'id_persona', how = 'left')
df = df.rename(columns = {'dosis' : 'dosis_2', 
                          'fecha_vacuna': 'fecha_vacuna_2', 
                          'id_vacuna': 'id_vacuna_2'})
df['dosis_2'].replace(2, 1, inplace = True)

In [123]:
# Agregar información de dosis 3
df = df.merge(df_vac[df_vac.dosis == 3], on = 'id_persona', how = 'left')
df = df.rename(columns = {'dosis' : 'dosis_3', 
                          'fecha_vacuna': 'fecha_vacuna_3', 
                          'id_vacuna': 'id_vacuna_3'})
df['dosis_3'].replace(3, 1, inplace = True)

In [124]:
# Verificamos que existen algunos duplicados al momento de añadir info de vacunación así que los eliminamos
len(df[df.id_persona.duplicated() == True])

27

In [125]:
# Eliminamos los duplicados
df = df.drop_duplicates('id_persona', keep='last') \
       .sort_values(by = ['tiempo_fal']) \
       .reset_index(drop = True)

In [126]:
# TOTAL de casos positivos más información de fallecimiento, más información de vacunación
len(df)

3381496

In [127]:
print(f'Personas con 1 dosis recibida en el dataset de casos positivos: {len(df[df.dosis_1 == 1])}')
print(f'Personas con 2 dosis recibidas en el dataset de casos positivos: {len(df[df.dosis_2 == 1])}')
print(f'Personas con 3 dosis recibidas en el dataset de casos positivos: {len(df[df.dosis_3 == 1])}')

Personas con 1 dosis recibida en el dataset de casos positivos: 3005371
Personas con 2 dosis recibidas en el dataset de casos positivos: 2932238
Personas con 3 dosis recibidas en el dataset de casos positivos: 2429882


In [128]:
df.head()

,id_persona,fecha_resultado,metododx,sexo,edad,edad_cat,departamento,fallecido,fecha_fallecimiento,fecha_fal,...,semana_fal,dosis_1,fecha_vacuna_1,id_vacuna_1,dosis_2,fecha_vacuna_2,id_vacuna_2,dosis_3,fecha_vacuna_3,id_vacuna_3
0,36527082,NaN,NaN,1,63,5,22,1,20200303.0,2020-03-03,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,36401995,NaN,NaN,1,85,7,22,1,20200304.0,2020-03-04,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,36853977,NaN,NaN,2,24,1,16,1,20200307.0,2020-03-07,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,14179222,NaN,NaN,2,76,6,8,1,20200308.0,2020-03-08,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,38353395,NaN,NaN,1,81,7,15,1,20200311.0,2020-03-11,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 2.7.3 Rellenar espacios vacíos

In [129]:
# Crear una columna llamado 'vacunado' si recibe 2 o 3 dosis
df['vacunado'] = np.where((df['dosis_2'] == 1) | (df['dosis_3'] == 1), 1, 0).astype('int8')

In [130]:
# Convertir de float a int para ocupar menos espacio en memoria
df['dosis_1'] = df['dosis_1'].fillna(0).astype('int8')
df['dosis_2'] = df['dosis_2'].fillna(0).astype('int8')
df['dosis_3'] = df['dosis_3'].fillna(0).astype('int8')

df['id_vacuna_1'] = df['id_vacuna_1'].fillna(0).astype('int8')
df['id_vacuna_2'] = df['id_vacuna_2'].fillna(0).astype('int8')
df['id_vacuna_3'] = df['id_vacuna_3'].fillna(0).astype('int8')

In [131]:
df.head()

,id_persona,fecha_resultado,metododx,sexo,edad,edad_cat,departamento,fallecido,fecha_fallecimiento,fecha_fal,...,dosis_1,fecha_vacuna_1,id_vacuna_1,dosis_2,fecha_vacuna_2,id_vacuna_2,dosis_3,fecha_vacuna_3,id_vacuna_3,vacunado
0,36527082,NaN,NaN,1,63,5,22,1,20200303.0,2020-03-03,...,0,NaN,0,0,NaN,0,0,NaN,0,0
1,36401995,NaN,NaN,1,85,7,22,1,20200304.0,2020-03-04,...,0,NaN,0,0,NaN,0,0,NaN,0,0
2,36853977,NaN,NaN,2,24,1,16,1,20200307.0,2020-03-07,...,0,NaN,0,0,NaN,0,0,NaN,0,0
3,14179222,NaN,NaN,2,76,6,8,1,20200308.0,2020-03-08,...,0,NaN,0,0,NaN,0,0,NaN,0,0
4,38353395,NaN,NaN,1,81,7,15,1,20200311.0,2020-03-11,...,0,NaN,0,0,NaN,0,0,NaN,0,0


## 3. Verificar errores

Se procede a verificar que no hayan errores en las fechas. Por ejemplo dosis recibidas luego de reportarse una fecha de fallecimiento.

In [132]:
# Se crea nueva columnas formato fecha de las columnas de vacunación
lst_dates = ['fecha_vacuna_1', 'fecha_vacuna_2', 'fecha_vacuna_3']

for date in lst_dates:
    df[date + '_'] = pd.to_datetime(df[date], format = '%Y%m%d')

In [133]:
# Cola del dataset principal, con 'fecha_fal', 'fecha_vacuna_1_', 'fecha_vacuna_2_', 'fecha_vacuna_3_'
df[['fecha_fal', 'fecha_vacuna_1_', 'fecha_vacuna_2_', 'fecha_vacuna_3_']].tail()

,fecha_fal,fecha_vacuna_1_,fecha_vacuna_2_,fecha_vacuna_3_
3381491,NaT,2021-11-05,2021-12-03,2022-03-09
3381492,NaT,NaT,NaT,NaT
3381493,NaT,2021-09-05,2021-09-26,2022-01-05
3381494,NaT,2021-08-27,2021-09-17,2021-12-30
3381495,NaT,2021-09-17,2021-10-08,NaT


In [134]:
# Casos donde la fecha de fallecimiento está ANTES de la fecha de vacunación
df[['fecha_fal', 'fecha_vacuna_1_', 'fecha_vacuna_2_', 'fecha_vacuna_3_']] \
[(df['fecha_vacuna_1'] >= df['fecha_fallecimiento']) |
 (df['fecha_vacuna_2'] >= df['fecha_fallecimiento']) |
 (df['fecha_vacuna_3'] >= df['fecha_fallecimiento']) ]

,fecha_fal,fecha_vacuna_1_,fecha_vacuna_2_,fecha_vacuna_3_
9,2020-03-16,2021-10-10,2021-10-31,NaT
116,2020-03-29,2021-03-09,2021-03-30,2021-12-09
222,2020-04-03,2021-02-11,2021-03-04,2021-11-03
236,2020-04-03,2021-12-04,2021-12-26,2022-03-28
241,2020-04-03,2021-08-28,2021-09-19,2022-05-04
...,...,...,...,...
206687,2022-02-13,2021-06-25,2021-07-20,2022-02-14
206702,2022-02-13,2022-01-22,2022-02-14,NaT
208016,2022-02-26,2021-11-10,2021-12-09,2022-03-26
208964,2022-03-13,2021-06-18,2021-07-09,2022-05-02


**EVIDENTEMENTE** una persona no puede recibir ninguna dosis de vacunación luego de ser reportado como fallecido por obvias razones. Se asume que en estos registros ocurrió algún problema, por lo que se procede a eliminarlos.

In [135]:
# Eliminar todos los registros erróneos donde la fecha de vacunación es mayor a la de fallecimiento
df.drop(df.index[(df['fecha_vacuna_1'] >= df['fecha_fallecimiento']) |
 (df['fecha_vacuna_2'] >= df['fecha_fallecimiento']) |
 (df['fecha_vacuna_3'] >= df['fecha_fallecimiento']) ], inplace = True)

df.reset_index(drop = True)

# Ordenar el dataframe final
df = df[['id_persona', 'fecha_resultado', 'metododx', 'sexo', 'edad', 'edad_cat',
       'departamento', 'fallecido', 'fecha_fallecimiento', 'fecha_fal', 'year',
       'epi_week', 'tiempo_fal', 'vacunado', 'dosis_1', 'fecha_vacuna_1', 'id_vacuna_1',
       'dosis_2', 'fecha_vacuna_2', 'id_vacuna_2', 'dosis_3', 'fecha_vacuna_3',
       'id_vacuna_3']]

In [136]:
# Cola del dataset final de casos positivos + información de fallecimiento + información de vacunación
df.tail()

,id_persona,fecha_resultado,metododx,sexo,edad,edad_cat,departamento,fallecido,fecha_fallecimiento,fecha_fal,...,vacunado,dosis_1,fecha_vacuna_1,id_vacuna_1,dosis_2,fecha_vacuna_2,id_vacuna_2,dosis_3,fecha_vacuna_3,id_vacuna_3
3381491,27997090,20220514.0,2.0,1,25,1,18,0,NaN,NaT,...,1,1,20211105.0,1,1,20211203.0,1,1,20220309.0,5
3381492,10904655,20220514.0,2.0,2,37,2,15,0,NaN,NaT,...,0,0,NaN,0,0,NaN,0,0,NaN,0
3381493,23465806,20220514.0,3.0,1,32,2,15,0,NaN,NaT,...,1,1,20210905.0,6,1,20210926.0,6,1,20220105.0,5
3381494,31853324,20220514.0,2.0,1,37,2,15,0,NaN,NaT,...,1,1,20210827.0,6,1,20210917.0,6,1,20211230.0,5
3381495,20941978,20220514.0,2.0,1,20,1,8,0,NaN,NaT,...,1,1,20210917.0,6,1,20211008.0,6,0,NaN,0


In [137]:
# TOTAL de casos en el dataset final
len(df)

3379653

In [138]:
# Verificamos columnas y data types
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3379653 entries, 0 to 3381495
Data columns (total 23 columns):
 #   Column               Dtype         
---  ------               -----         
 0   id_persona           int32         
 1   fecha_resultado      float64       
 2   metododx             float64       
 3   sexo                 int8          
 4   edad                 int8          
 5   edad_cat             int8          
 6   departamento         int8          
 7   fallecido            int8          
 8   fecha_fallecimiento  float64       
 9   fecha_fal            datetime64[ns]
 10  year                 int16         
 11  epi_week             int8          
 12  tiempo_fal           float64       
 13  vacunado             int8          
 14  dosis_1              int8          
 15  fecha_vacuna_1       float64       
 16  id_vacuna_1          int8          
 17  dosis_2              int8          
 18  fecha_vacuna_2       float64       
 19  id_vacuna_2          

In [139]:
# Verificamos valores faltantes
df.isnull().sum()

id_persona                   0
fecha_resultado         106244
metododx                106244
sexo                         0
edad                         0
edad_cat                     0
departamento                 0
fallecido                    0
fecha_fallecimiento    3171130
fecha_fal              3171130
year                         0
epi_week                     0
tiempo_fal             3171130
vacunado                     0
dosis_1                      0
fecha_vacuna_1          376105
id_vacuna_1                  0
dosis_2                      0
fecha_vacuna_2          448866
id_vacuna_2                  0
dosis_3                      0
fecha_vacuna_3          950892
id_vacuna_3                  0
dtype: int64

## 4. Guardar el df

In [140]:
# Guardamos en un csv
file_name = 'DP3_covid19-peru_casos-positivos_+_info-fallecimiento_+_info-vacunas.csv'
df.to_csv(file_name,  index=False)